1. Дан список data. Напишите функцию, которая возвращает сумму элементов на диагонали. Т. е. 13+32+23+35.

In [3]:
def get_diagonal_sum(matrix):
    i = 0
    diagonal_sum = 0
    for row in matrix:
        diagonal_sum+=row[i]
        i+=1
    return diagonal_sum 
       
data = [
    [13, 25, 23, 34],
    [45, 32, 44, 47],
    [12, 33, 23, 95],
    [13, 53, 34, 35],
]

get_diagonal_sum(data)

103

2. Дан список чисел, часть из которых имеют строковый тип или содержат буквы. Напишите функцию, которая возвращает сумму квадратов элементов, которые могут быть числами.

In [7]:
def get_sum_of_squares(elements):
    result = 0
    for e in elements:
        try:
            result+=int(e)
        except ValueError:
            continue
    return result

data = [1, '5', 'abc', 20, '2']
get_sum_of_squares(data)

28

3. Напишите функцию, которая возвращает название валюты (поле 'Name') с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

In [20]:
import urllib.request, json
from operator import itemgetter

def get_max_currency_exchange_rate():
    with urllib.request.urlopen(r'https://www.cbr-xml-daily.ru/daily_json.js') as url:
        currencies = json.loads(url.read().decode())
        
    return max(currencies['Valute'].values(), key=itemgetter('Value', 'Name'))['Name']

get_max_currency_exchange_rate()

'Китайских юаней'

4.1. Добавьте в класс еще один формат, который возвращает название валюты (например, 'Евро').

In [40]:
import requests

In [44]:
class Rate:
    def __init__(self, format='value'):
        self.format = format
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в трех вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        - курс валюты при self.format == 'value'
        Rate('value').make_format('EUR')
        79.4966
        
        - название валюты при self.format == 'name'
        Rate('name').make_format('EUR')
        'Евро'
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                return response[currency]['Value']
            
            if self.format == 'name':
                return response[currency]['Name']
        
        return 'Error'
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')

In [45]:
Rate(format='name').eur()

'Евро'

4.2. Добавьте в класс параметр diff (со значениями True или False), который в случае значения True в методах eur и usd будет возвращать не курс валюты, а изменение по сравнению в прошлым значением.

In [89]:
class Rate:
    def __init__(self, format='value', diff=False):
        self.format = format
        self.diff = diff
        #чтобы не падать с exception при попытке работать не с полным форматом
        if diff:
            self.format = 'full'
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в трех вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        - курс валюты при self.format == 'value'
        Rate('value').make_format('EUR')
        79.4966
        
        - название валюты при self.format == 'name'
        Rate('name').make_format('EUR')
        'Евро'
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                return response[currency]['Value']
            
            if self.format == 'name':
                return response[currency]['Name']
        
        return 'Error'
    
    def eur(self):
        """
        Возвращает курс евро на сегодня в формате self.format.
        Если self.diff == True, вернет разницу курсов вчерашенго и сегодняшнего.
        """
        if self.diff:
            return round(self.make_format('EUR')['Value'] - self.make_format('EUR')['Previous'], 4)
        
        return self.make_format('EUR')
    
    def usd(self):
        """
        Возвращает курс доллара на сегодня в формате self.format.
        Если self.diff == True, вернет разницу курсов вчерашенго и сегодняшнего.
        """
        if self.diff:
            return round(self.make_format('USD')['Value'] - self.make_format('USD')['Previous'], 4)
        
        return self.make_format('USD')

In [91]:
Rate(diff=True).eur()

-0.4029

In [92]:
Rate(diff=True).usd()

-0.0797

5. Напишите функцию, возвращающую сумму первых n чисел Фибоначчи

In [36]:
def fibonacci(n):
    if n == 0 or n == 1:
        return n
    else:
        return fibonacci(n-2) + fibonacci(n-1)

def sum_fibonacci(n):
    return sum(fibonacci(i) for i in range(n))

sum_fibonacci(10)

88

6. Напишите функцию, преобразующую произвольный список вида ['2018-01-01', 'yandex', 'cpc', 100] в словарь {'2018-01-01': {'yandex': {'cpc': 100}}}

In [24]:
def list_to_nested_dicts(list):
    if len(list) == 1:
        return list[0]
    else:
        return {list[0]:list_to_nested_dicts(list[1:])}

list_to_nested_dicts(['2018-01-01', 'yandex', 'cpc', 100])

{'2018-01-01': {'yandex': {'cpc': 100}}}